<a href="https://colab.research.google.com/github/wksmirnowa/compling_homeworks/blob/master/HW08_Keyword_extraction_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Извлечение ключевых слов

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Импорты и предобработка данных

In [ ]:
!pip install pymorphy2[fast]

In [ ]:
import json, os, re
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from string import punctuation
import numpy as np
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()

In [ ]:
!CFLAGS="-Wno-narrowing" pip install cld2-cffi

In [ ]:
!pip install multi-rake

In [ ]:
!pip install rake_nltk

In [ ]:
!pip install rutermextract

In [ ]:
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
added_stops = {'весь', 'это', 'наш', 'оно', 'итак', 'т.п', 'т.е', 'мало', 'меньше', 'ещё', 'слишком', 'также',
                   'ваш', 'б', 'хм', 'который', 'свой', 'не', 'мочь', 'однако', 'очень', 'благодаря', 'кроме', 'е'}

stops = stops.union(added_stops)
stops_en = set(stopwords.words('english'))
stops = stops.union(stops_en)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
      words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
      words = [morph.parse(word)[0] for word in words if word and word not in stops]
      words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

      return " ".join(words)

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
PATH_TO_DATA = '/content/drive/My Drive/data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [ ]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [ ]:
%%time
data['norm_content'] = data['content'].apply(lambda x: normalize(x))

CPU times: user 7min 8s, sys: 249 ms, total: 7min 8s
Wall time: 7min 8s


In [ ]:
data.head(10)

,keywords,title,url,abstract,content,summary,norm_content
0,"[ИЕРАРХИЧЕСКИЕ ИГРЫ, МАКСИМАЛЬНЫЙ ГАРАНТИРОВАННЫЙ РЕЗУЛЬТАТ, УСТОЙЧИВОСТЬ]",Об одной гипотезе в основаниях теории иерархических игр,https://cyberleninka.ru/article/n/ob-odnoy-gipoteze-v-osnovaniyah-teorii-ierarhicheskih-igr,"При исследовании игр с фиксированным порядком ходов обычно принимается некоторая гипотеза о поведении игрока нижнего уровня, существенно упрощающая как «ответ» в получающейся задаче, так и рассуждения, приводящие к этому ответу. Ниже обсуждается адекватность этой гипотезы. При этом рассматривается простейшая игра двух лиц, на примере которой видны как основные проблемы, так и некоторые подходы к их решению.In the study of hierarchical games a hypothesis on the behavior of the low-level player is normally accepted. It simplifies sufficiently both the solution of the problem in hand and its reasoning. The adequacy of this hypothesis is discussed below. The simplest two-player game is used to illustrate both the major problems and some approaches to their solution.","Системный анализ\nУДК 519.865 + 519.95\nББК 22.165\n\nОБ ОДНОЙ ГИПОТЕЗЕ В ОСНОВАНИЯХ ТЕОРИИ\nИЕРАРХИЧЕСКИХ ИГР\nГорелов М. А. 1\n(Вычислительный центр РАН, Москва)\nПри исследовании игр с фиксированным порядком ходов обычно принимается некоторая гипотеза о поведении игрока нижнего уровня, существенно упрощающая как «ответ» в получающейся задаче, так и рассуждения, приводящие к этому\nответу. Ниже обсуждается адекватность этой гипотезы.\nПри этом рассматривается простейшая игра двух лиц, на\nпримере которой видны как основные проблемы, так и некоторые подходы к их решению.\n\nВведение\nПусть имеется игра двух лиц, в которой первый игрок выбирает свои управления из множества U, а второй – из множества V. Будем считать, что цели игроков описываются стремлением к максимизации функций g и h соответственно.\nРассмотрим следующую схему взаимодействия игроков (по\nтрадиции носящей имя игры Γ2 [3]). Предположим, что первый\nигрок рассчитывает и действительно будет иметь информацию о\nвыборе...",NaN,анализ гипотеза основание теория игра горелов метр центр рана москва исследование игра ход гипотеза поведение игрок уровень ответ задача рассуждение ответ адекватность гипотеза игра лицо пример проблема подход решение введение игра лицо игрок управление множество множество цель игрок стремление максимизация функция схема взаимодействие игрок традиция имя игра игрок информация выбор партнёр управление принятие решение образ стратегия первое михаил александр горелов кандидат наука управление система выпуск игрок функция множество множество множество стратегия игрок стратегия партнёр ситуация первое игрок ответ вопрос принцип работа принцип эффективность стратегия основа получение величина выигрыш принцип исследователь операция информация основа принцип информация функция выигрыш игрок оценка множество ответ стратегия игрок партнёр стратегия результат решение точка максимум критерий минимум вопрос вопрос поведение игрок максимум функция точка вопрос игрок реализация максимум выигрыш в...
1,"[ЧЕРНОБЫЛЬСКАЯ АЭС, авария, ликвидация последствий, средства защиты, Chernobyl Nuclear Power Station, Accident, liquidation of consequences, protection means]",Научно-методическое сопровождение работ по ликвидации последствий аварии на Чернобыльской АЭС,https://cyberleninka.ru/article/n/nauchno-metodicheskoe-soprovozhdenie-rabot-po-likvidatsii-posledstviy-avarii-na-chernobylskoy-aes,В статье кратко изложена деятельность сотрудников Всесоюзного научно-исследовательского института гражданской обороны по на .Article shows the activity of employees of All-Union scientific research institute of civil defence on the consequences liquidation.,"/4\n\nCivil SecurityTechnology, Vol. 8, 2011, No. 2 (28)\n\nУДК 621.039.586\n\nНаучно-методическое сопровождение работ\nпо ликвидации последствий аварии\nна Чернобыльской АЭС\nISSN 1996-8493\n© Технологии гражданской безопасности, 2011\n\nБ.А. Галушкин, С.В. Горбунов, В.С. Исаев, В.Н. 

In [ ]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

## Бейзлайн

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [ ]:
tfidf.fit(data['norm_content'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [ ]:
texts_vectors = tfidf.transform(data['norm_content'])

In [ ]:
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [ ]:
keywords[:3]

[['второе игрок',
  'игрок',
  'первое игрок',
  'игра второе',
  'игра',
  'выигрыш',
  'стратегия',
  'множество',
  'игрок игра',
  'второе'],
 ['авария',
  'последствие авария',
  'аэс',
  'внии',
  'последствие',
  'авария аэс',
  'ликвидация последствие',
  'ликвидация',
  'дезактивация',
  'центр'],
 ['фонд',
  'библиотека',
  'хранение фонд',
  'хранение',
  'система',
  'перераспределение',
  'формирование фонд',
  'взаимодействие библиотека',
  'репозитарий',
  'система хранение']]

In [ ]:
evaluate(data['keywords'], keywords)

Precision -  0.09
Recall -  0.12
F1 -  0.09
Jaccard -  0.05


**Бейзлайн на всем датасете:** \
Precision -  0.09 \
Recall -  0.12 \
F1 -  0.09 \
Jaccard -  0.05

In [ ]:
data.dropna()

## Эксперимент 1.

Генсимовский извлекатор ключевых слов, как мы знаем, бейзлайн не преодолевает. Поэтому я решила попробовать найти другие готовые библиотеки для извлечения ключевых слов и проверить, смогут ли они побить бейзлайн. \

1. [Rake](https://pypi.org/project/rake-nltk/) – как я поняла, использует в своей основе NLTK, но неясно, связан ли он как-то с разработчиками NLTK. Он не совсем подходит для русскоязычных данных, поскольку написан для работы с английскими текстами. Тем не менее он побил бейзлайн по полноте. 
2. [multi_rake](https://pypi.org/project/multi-rake/) – аналог Rake от русскоязычного разработчика; multi_rake поддерживает сразу несколько языков, в том числе русский и английский. Несмотря на этот факт, он сработал хуже всех.
3. [rutermextract](https://pypi.org/project/rutermextract/) – извлекатор важных слов, написанный с использованием правил специально для русского языка. Вот он смог преодолеть бейзлайн по всем метрикам, пусть и чуть-чуть. 

In [ ]:
%%time
from rake_nltk import Rake
r = Rake()

def rake(line):
  r.extract_keywords_from_text(line)
  keywords = r.get_ranked_phrases()
  if keywords == [] or keywords is None:
    return 'None'
  else:
    return keywords[0].split()

keywords_rake = data['norm_content'].apply(lambda x: rake(x))

CPU times: user 6min 31s, sys: 537 ms, total: 6min 31s
Wall time: 6min 31s


In [ ]:
keywords_rake[:10]

0                                                             [рациональность, вариант, предположение, значение, задача, второе, игрок, том, максимум, функция, решение, задача, второе, игрок, решение, задача, игра, противник, ожидание, вопрос, модель, противник, идеализация, случай, игра, второе, игрок, постановка, вопрос, случай, максимум, задача, игрок, игра, случай, предположение, компактность, множество, управление, непрерывность, функция, выигрыш, игрок, предположение, задача, метод, анализ, случай, топология, множество, стратегия, сантиметр, повод, путь, структура, стратегия, термин, множество, топология, вопрос, модель, обмен, информация, действие, игрок, проблема, образ, ограничение, функционал, значение, результат, первое, игрок, игра, противник, структура, стратегия, оценка, управление, система, выпуск, рассмотрение, частное, случай, люба, стратегия, значение, число, игрок, поведение, стратегия, неравенство, результат, первое, игрок, игра, противник, результат, игра, второе, 

In [ ]:
evaluate(data['keywords'], keywords_rake) #полнота повыше

Precision -  0.02
Recall -  0.23
F1 -  0.03
Jaccard -  0.02


In [ ]:
%%time
from multi_rake import Rake
r = Rake(max_words=5, language_code='ru')

def multirake(line):
  keywords = r.apply(line, text_for_stopwords=stops)
  if keywords == [] or keywords is None:
    return ' '
  else:
    for k in keywords:
      return k[0].split()

keywords_multirake = data['norm_content'].apply(lambda x: multirake(x))

CPU times: user 32.3 s, sys: 70.9 ms, total: 32.3 s
Wall time: 32.3 s


In [ ]:
keywords_multirake[:10]

0    [множество, стратегия, семейство, подмножество, множество]
1              [директор, ликвидатор, последствие, авария, аэс]
2               [-практ, конф, новосибирск, материал, редакция]
3                          [результат, план, развитие, команда]
4        [машина, изобретение, цель, двигатель, промышленность]
5                        [кільцевих, кут, угненко, професор, д]
6                                    [модель, климат, сценарий]
7                                [число, миллион, веб-страница]
8             [испытание, прибор, привод, метр, машиностроение]
9                       [утечка, вещество, аэс, течение, месяц]
Name: norm_content, dtype: object

In [ ]:
evaluate(data['keywords'], keywords_multirake)

Precision -  0.03
Recall -  0.02
F1 -  0.02
Jaccard -  0.01


In [ ]:
%%time
from rutermextract import TermExtractor
term_extractor = TermExtractor()

def term(line):
  keywords_term = []
  for te in term_extractor(line, limit=10, nested=True):
    keywords_term.append(te.normalized)
  return keywords_term

keywords_term = data['norm_content'].apply(lambda x: term(x))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


CPU times: user 12min 27s, sys: 186 ms, total: 12min 27s
Wall time: 12min 28s


In [ ]:
keywords_term[:10]

0           [игрок, игра, множество, стратегия, результат, второе игрок, первое, выигрыш, точка, функция]
1                   [работа, авария, центр, год, последствие, аэс, ссср, исследование, ликвидация, пункт]
2      [фонд, система, библиотека, хранение, формирование, документ, перераспределение, год, этап, форма]
3    [коммуникация, команда, сообщество, система, проект, форма, развитие, работа, организация, общество]
4    [наука, производство, развитие, техника, этап, машина, основа, двигатель, революция, машиностроение]
5                                   [дорога, обход, міст, кільцевих, кут, число, ы, угненко, рис, радіус]
6                 [модель, осадки, водосбор, год, озеро, климат, сценарий, сумма осадок, условие, период]
7                       [документ, система, поиск, запрос, слово, ипс, оператор, язык, яндекс, результат]
8                  [редуктор, испытание, контроль, ход, передача, ресурс, мёртвый, вала, параметр, износ]
9                                   [аэс, тече

In [ ]:
evaluate(data['keywords'], keywords_term)

Precision -  0.1
Recall -  0.13
F1 -  0.1
Jaccard -  0.06


**Бейзлайн**         \    **Метрики TermExtractor** \
Precision - 0.09  \    Precision -  0.1\
Recall - 0.12     \    Recall - 0.13\
F1 - 0.09         \    F1 - 0.1\
Jaccard - 0.05    \    Jaccard - 0.06

Вывод: по всем метрикам TermExtractor ненамного, но всё же превзошел бейзлайн

## Эксперимент 2.

Я подумала, что можно попробовать побить бейзлайн, подобрав подходящие значения для векторайзера. Почему-то модель без нграмм и с самой минимальной частотой встречаемости слов сработала лучше, чем модель с биграммами и минимальным значением встречаемости слов 5. Добавление максимального значения встречаемости слова и ограничение по числу признаков почему-то тоже не помогали преодолеть бейзлайн (пример ниже). Возможно, если сравнить эти модели, то с человеческой точки зрения качество модели с нграммами повысится, но по метрикам получается так.

In [ ]:
tfidf = TfidfVectorizer(min_df=1, ngram_range=(1,1))
tfidf.fit(data['norm_content'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['norm_content'])
keywords = []
for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [ ]:
keywords[:3]

[['игрок',
  'игра',
  'выигрыш',
  'стратегия',
  'множество',
  'второе',
  'первое',
  'лемма',
  'результат',
  'функция'],
 ['авария',
  'аэс',
  'внии',
  'последствие',
  'ликвидация',
  'дезактивация',
  'центр',
  'ссср',
  'чаэс',
  'полигон'],
 ['фонд',
  'библиотека',
  'хранение',
  'система',
  'перераспределение',
  'грихан',
  'репозитарий',
  'депозитарий',
  'носб',
  'гпнтба']]

In [ ]:
evaluate(data['keywords'], keywords)

Precision -  0.1
Recall -  0.13
F1 -  0.1
Jaccard -  0.06


**Бейзлайн**         \    **Метрики TFIDF с другими параметрами** \
Precision - 0.09  \    Precision -  0.1\
Recall - 0.12     \    Recall - 0.13\
F1 - 0.09         \    F1 - 0.1\
Jaccard - 0.05    \    Jaccard - 0.06

Бейзлайн снова побили, но совсем чуть-чуть – ровно так же, как и с TermExtractor. Ниже – пример того, что дополнительные параметры преодолеть бейзлайн не помогают.

In [ ]:
#пример неудачно сработавшей модели
tfidf = TfidfVectorizer(max_df=500, min_df=1, ngram_range=(1,2), max_features=500)
tfidf.fit(data['norm_content'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['norm_content'])
keywords = []
for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1] #было[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [ ]:
keywords[:3]

[['управление система',
  'теорема',
  'гипотеза',
  'равенство',
  'неравенство',
  'константа',
  'учебник',
  'неопределённость',
  'академия наука',
  'йорк'],
 ['авария',
  'загрязнение',
  'белоруссия',
  'отход',
  'вс',
  'выброс',
  'реактор',
  'грунт',
  'растение',
  'леса'],
 ['бюллетень',
  'экземпляр',
  'хранилище',
  'система система',
  'округа',
  'парадигма',
  'академия наука',
  'новосибирск',
  'указатель',
  'управление система']]

In [ ]:
evaluate(data['keywords'], keywords)

Precision -  0.03
Recall -  0.03
F1 -  0.03
Jaccard -  0.01


## Эксперимент 3.

Я попробовала использовать метод графов: подобрать новые меры центральности и поэкспериментировать с ними.

В результате по каким-то странным причинам разные меры центральности дали одинаковый результат: побили бейзлайн по точности, f-мере и Жаккару, но показали при этом одинаковые цифры.

In [ ]:
import networkx as nx
import itertools
from itertools import combinations

In [ ]:
def build_matrix(text, window_size=5):
  vocab_list = text.split()
  vocab = set(vocab_list)
  word2id = {w:i for i, w in enumerate(vocab)}
  id2word = {i:w for i, w in enumerate(vocab)}
  ids = [word2id[word] for word in vocab_list]

  m = np.zeros((len(vocab), len(vocab)))

  for i in range(0, len(ids), window_size):
    window = ids[i:i+window_size]
    for j, k in combinations(window, 2):
      m[j][k] += 1
      m[k][j] += 1
    
  return m, id2word

#для сравнения – основная метрика центральности
def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.degree_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
keyword_nx = data['norm_content'].apply(lambda x: some_centrality_measure(x))

In [ ]:
keyword_nx[:10]

0                  [игра, игрок, множество, результат, стратегия]
1                      [работа, центр, авария, год, исследование]
2             [фонд, система, библиотека, хранение, формирование]
3    [коммуникация, команда, сообщество, организация, технология]
4             [наука, производство, развитие, техника, двигатель]
5                           [обход, дорога, кільцевих, міст, кут]
6                     [модель, осадок, водосбор, сценарий, озеро]
7                         [документ, система, поиск, запрос, ипс]
8                  [редуктор, испытание, контроль, ход, передача]
9                              [аэс, течение, океан, остров, рис]
Name: norm_content, dtype: object

In [ ]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.09
F1 -  0.1
Jaccard -  0.06


**Бейзлайн**         \    **Метрики networkx** \
Precision - 0.09  \    Precision -  0.13\
Recall - 0.12     \    Recall - 0.09\
F1 - 0.09         \    F1 - 0.1\
Jaccard - 0.05    \    Jaccard - 0.06

Здесь я попробовала использовать метрику центральности узла, основанную на степени близости узла. Моя идея состояла в том, что самое распространенное и важное слово должно оказаться самым близким к большинству слов. Именно это и показывает данная метрика. Итог тот же.

In [ ]:
def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.closeness_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
%%time
keyword_nx1 = data['norm_content'].apply(lambda x: some_centrality_measure(x))

CPU times: user 1h 6min 41s, sys: 1.34 s, total: 1h 6min 42s
Wall time: 1h 6min 45s


In [ ]:
keyword_nx1[:10]

0               [игра, игрок, множество, стратегия, первое]
1                [работа, авария, год, исследование, центр]
2       [фонд, система, библиотека, хранение, формирование]
3    [команда, коммуникация, сообщество, форма, технология]
4           [наука, производство, развитие, машина, основа]
5              [дорога, кільцевих, обход, міст, перевезень]
6              [модель, осадок, водосбор, климат, сценарий]
7                   [документ, система, поиск, запрос, ипс]
8                [редуктор, испытание, контроль, вала, ход]
9                       [аэс, океан, течение, остров, вода]
Name: norm_content, dtype: object

In [ ]:
evaluate(data['keywords'], keyword_nx1)

Precision -  0.13
Recall -  0.09
F1 -  0.1
Jaccard -  0.06


**Бейзлайн**         \    **Метрики networkx** \
Precision - 0.09  \    Precision -  0.13\
Recall - 0.12     \    Recall - 0.09\
F1 - 0.09         \    F1 - 0.1\
Jaccard - 0.05    \    Jaccard - 0.06

Также я решила попробовать применить степень посредничества. По идее ключевые слова могут оказаться между большинством остальных слов, поэтому можно попробовать использовать такую метрику центральности узла и посмотреть, поможет ли она найти ключевые слова верно. Результат такой же.

In [ ]:
def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.betweenness_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
%%time
keyword_nx2 = data['norm_content'].apply(lambda x: some_centrality_measure(x))

CPU times: user 57min 26s, sys: 563 ms, total: 57min 27s
Wall time: 57min 29s


In [ ]:
keyword_nx2[:10]

0                           [игра, игрок, множество, метр, том]
1                    [год, работа, центр, авария, исследование]
2                    [фонд, система, библиотека, год, хранение]
3    [коммуникация, команда, организация, сообщество, общество]
4             [наука, производство, машина, двигатель, техника]
5                      [дорога, обход, кільцевих, фактор, міст]
6                   [модель, осадок, озеро, водосбор, сценарий]
7                       [документ, система, поиск, ипс, запрос]
8                 [редуктор, испытание, контроль, момент, вала]
9                           [аэс, течение, океан, остров, вода]
Name: norm_content, dtype: object

In [ ]:
evaluate(data['keywords'], keyword_nx2)

Precision -  0.13
Recall -  0.09
F1 -  0.1
Jaccard -  0.06


**Бейзлайн**         \    **Метрики networkx** \
Precision - 0.09  \    Precision -  0.13\
Recall - 0.12     \    Recall - 0.09\
F1 - 0.09         \    F1 - 0.1\
Jaccard - 0.05    \    Jaccard - 0.06

А вот [тут](https://www.searchenginejournal.com/harmonic-centrality-pagerank/283985/#close) есть  целый материал про еще одну метрику – Harmonic centrality. Ее сравнивают с Page Rank. Попробуем применить и ее и посмотреть, насколько она будет эффективна для нас

In [ ]:
def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.harmonic_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
%%time
keyword_nx3 = data['norm_content'].apply(lambda x: some_centrality_measure(x))

CPU times: user 1h 9min 13s, sys: 1.38 s, total: 1h 9min 14s
Wall time: 1h 9min 17s


In [ ]:
keyword_nx3[:10]

0            [игра, игрок, множество, стратегия, результат]
1                [работа, авария, центр, год, исследование]
2       [фонд, система, библиотека, хранение, формирование]
3    [коммуникация, команда, сообщество, технология, форма]
4          [наука, производство, развитие, техника, машина]
5                     [дорога, обход, кільцевих, міст, кут]
6              [модель, осадок, водосбор, климат, сценарий]
7                   [документ, система, поиск, запрос, ипс]
8            [редуктор, испытание, контроль, ход, передача]
9                        [аэс, течение, океан, остров, рис]
Name: norm_content, dtype: object

In [ ]:
evaluate(data['keywords'], keyword_nx3)

Precision -  0.13
Recall -  0.09
F1 -  0.1
Jaccard -  0.06


**Бейзлайн**         \    **Метрики networkx** \
Precision - 0.09  \    Precision -  0.13\
Recall - 0.12     \    Recall - 0.09\
F1 - 0.09         \    F1 - 0.1\
Jaccard - 0.05    \    Jaccard - 0.06

Вывод: видимо, нет особой разницы в том, какую метрику центральности использовать. Можно попробовать, например, увеличить метрики за счет увеличения количества выделяемых ключевых слов, но тогда мы выйдем за заданные для этого эксперимента ограничения. 